In [ ]:
# run against production 2024-11-06
from django.core.exceptions import ObjectDoesNotExist
from tqdm import tqdm
from dumper.utils import gsheet_to_df

In [ ]:
df = gsheet_to_df("1-_WXbdmpLzz_9vBiZ1y9v2tle_26m4tAhPTGzkkin5E")
col, _= Collection.objects.get_or_create(name="Schnitzler-Veranstaltungen")

In [ ]:
df.head(3)

In [ ]:
relation_type_lookup = {
    "1049": {
        "relation_type":PersonWork,
        "relation": PersonWorkRelation.objects.get(id=1049),
        "source_class": Person,
        "target_class": Work,
    },
    "1181": {
            "relation_type": PersonPlace,
            "relation": PersonPlaceRelation.objects.get(id=1181),
            "source_class": Person,
            "target_class": Place,
        },
    "1201": {
            "relation_type": EventWork,
            "relation": EventWorkRelation.objects.get(id=1201),
            "source_class": Event,
            "target_class": Work,
        },
    "1204": {
            "relation_type": EventWork,
            "relation": EventWorkRelation.objects.get(id=1204),
            "source_class": Event,
            "target_class": Work,
        },
    "1206": {
            "relation_type": EventWork,
            "relation": EventWorkRelation.objects.get(id=1206),
            "source_class": Event,
            "target_class": Work,
        },
    "1224": {
            "relation_type": PersonWork,
            "relation": PersonWorkRelation.objects.get(id=1224),
            "source_class": Person,
            "target_class": Work,
        },
    "1252": {
            "relation_type": EventWork,
            "relation": EventWorkRelation.objects.get(id=1252),
            "source_class": Event,
            "target_class": Work,
        },
    "1256": {
            "relation_type": PersonEvent,
            "relation": PersonEventRelation.objects.get(id=1256),
            "source_class": Person,
            "target_class": Event,
        },
    "1351": {
            "relation_type": InstitutionEvent,
            "relation": InstitutionEventRelation.objects.get(id=1351),
            "source_class": Institution,
            "target_class": Event,
        },
    "1369": {
            "relation_type": PlaceEvent,
            "relation": PlaceEventRelation.objects.get(id=1369),
            "source_class": Place,
            "target_class": Event,
        },
    "1369": {
            "relation_type": PlaceEvent,
            "relation": PlaceEventRelation.objects.get(id=1369),
            "source_class": Place,
            "target_class": Event,
        },
    "1370": {
            "relation_type": PersonEvent,
            "relation": PersonEventRelation.objects.get(id=1370),
            "source_class": Person,
            "target_class": Event,
        },
    "1527": {
            "relation_type": EventWork,
            "relation": EventWorkRelation.objects.get(id=1527),
            "source_class": Event,
            "target_class": Work,
        },
    "1528": {
            "relation_type": EventWork,
            "relation": EventWorkRelation.objects.get(id=1528),
            "source_class": Event,
            "target_class": Work,
        },
    "4362": {
            "relation_type": EventWork,
            "relation": EventWorkRelation.objects.get(id=4362),
            "source_class": Event,
            "target_class": Work,
        },
   
}

In [ ]:
does_not_exist = set()
for g, ndf in df.groupby("relation-id"):
    relation = relation_type_lookup[str(g)]["relation"]
    relation_class = relation_type_lookup[str(g)]["relation_type"]
    source_class = relation_type_lookup[str(g)]["source_class"]
    target_class = relation_type_lookup[str(g)]["target_class"]
    print(relation, relation_class, target_class)
    for i, row in tqdm(ndf.iterrows(), total=len(ndf)):
        try:
            source = source_class.objects.get(pk=int(row["source_id"]))
        except (ObjectDoesNotExist, ValueError):
            does_not_exist.add(row["source_id"])
            continue
        try:
            target = target_class.objects.get(pk=int(row["target_id"]))
        except ObjectDoesNotExist:
            does_not_exist.add(row["target_id"])
        if isinstance(row["relation_start_date_written"], str):
            start_date_written = row["relation_start_date_written"]
            end_date_written = row["relation_end_date_written"]
        else:
            start_date_written = None
            end_date_written = None
        relation_object = {
            relation_class.get_related_entity_field_namea(): source,
            relation_class.get_related_entity_field_nameb(): target,
            "relation_type": relation,
            "start_date_written": start_date_written,
            "end_date_written": end_date_written
        }
        created_relation, _ = relation_class.objects.get_or_create(**relation_object)
        created_relation.collection.add(col)
